In [1]:
import os
import sagemaker
import numpy as np
from sagemaker.tensorflow import TensorFlow
from tensorflow.python.keras.preprocessing.image import load_img

ON_SAGEMAKER_NOTEBOOK = True

sagemaker_session = sagemaker.Session()
if ON_SAGEMAKER_NOTEBOOK:
    role = sagemaker.get_execution_role()
else:
    role = "[YOUR ROLE]"

In [2]:
bucket = "surface-detection-dataset"

train_instance_type='ml.m4.xlarge'      # The type of EC2 instance which will be used for training
deploy_instance_type='ml.m4.xlarge'     # The type of EC2 instance which will be used for deployment

In [3]:
train_input_path = "s3://{}/train".format(bucket)
validation_input_path = "s3://{}/test".format(bucket)

In [4]:
train_input_path = sagemaker.s3_input(train_input_path)
validation_input_path = sagemaker.s3_input(validation_input_path)

In [5]:
hyperparameters={
    "learning_rate": 1e-3,
    "decay": 1e-7
}

In [6]:
estimator = TensorFlow(
  entry_point=os.path.join(os.path.abspath('.'), "baseline_model.py"),             # Your entry script
  role=role,
  framework_version="1.13.1",               # TensorFlow's version
  hyperparameters=hyperparameters,
  train_instance_count=1,                   # "The number of GPUs instances to use"
  train_instance_type=train_instance_type,
  script_mode=True,
  py_version="py3",
  model_dir='/opt/ml/model',
  base_job_name='surface-detect-cnn'
)

print(os.path.join(os.path.abspath('.'), "baseline_model.py"))

/home/ec2-user/SageMaker/surface-defect-cnn/baseline_model.py


In [ ]:
estimator.fit({'train': train_input_path, 'test': validation_input_path})

2019-11-19 20:45:09 Starting - Starting the training job...
2019-11-19 20:45:10 Starting - Launching requested ML instances...
2019-11-19 20:46:07 Starting - Preparing the instances for training............
2019-11-19 20:47:51 Downloading - Downloading input data...
2019-11-19 20:48:39 Training - Downloading the training image...
2019-11-19 20:48:57 Training - Training image download completed. Training in progress......2019-11-19 20:49:02,886 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-11-19 20:49:02,892 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-11-19 20:49:03,308 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-11-19 20:49:03,327 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-11-19 20:49:03,345 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-11-19 20:49:03,359 sagemaker-containers INFO     Invok

In [8]:
from baseline_model import keras_model_fn

model = keras_model_fn(hyperparameters)
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (Conv2D)              (None, 7500, 5144, 4)     104       
_________________________________________________________________
conv2d (Conv2D)              (None, 7496, 5140, 4)     404       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3748, 2570, 4)     0         
_________________________________________________________________
dropout (Dropout)            (None, 3748, 2570, 4)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1874, 1285, 4)     404       
_________________________________________________________________
conv2d_2 (Conv2D)    